In [1]:
!pip install qiskit
!pip install qiskit_aer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.0/8.0 MB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 21.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 93.9 MB/s eta 0:00:00


In [2]:
# Deutsch–Jozsa Algorithm using Qiskit 2.x
# Compatible with Qiskit >= 2.0.0

from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, transpile
from qiskit_aer import AerSimulator
from qiskit.visualization import plot_histogram
import matplotlib.pyplot as plt


In [3]:
# Task 1: Balanced oracle flipping ancilla for half of inputs (MSB == 0)
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, transpile
from qiskit_aer import AerSimulator

def oracle_flip_if_msb_zero(qc, inputs, ancilla):
    """
    Balanced oracle: flip ancilla when MSB (highest-index input) == 0.
    This covers exactly half of all inputs.
    Implementation: X on MSB (to invert 0->1), then CX from MSB to ancilla, then undo X.
    """
    msb = inputs[-1]
    qc.x(msb)               # convert MSB==0 -> 1 so control works
    qc.cx(msb, ancilla)     # flip ancilla when MSB was 0 originally
    qc.x(msb)               # undo

def deutsch_jozsa_circuit(n, oracle_func, oracle_args):
    qreg = QuantumRegister(n+1, "q")
    creg = ClassicalRegister(n, "c")
    qc = QuantumCircuit(qreg, creg)
    inputs = list(range(n))
    ancilla = n

    qc.x(ancilla)           # ancilla |1>
    qc.h(qreg)              # H on all (inputs + ancilla)
    oracle_func(qc, inputs, ancilla)  # apply oracle
    # H on input qubits
    qc.h(inputs)
    qc.measure(inputs, creg)
    return qc

# Run example for n=3
n = 3
qc = deutsch_jozsa_circuit(n, oracle_flip_if_msb_zero, None)
print(qc.draw(fold=-1))  # ASCII circuit (safe)
sim = AerSimulator()
counts = sim.run(transpile(qc, sim)).result().get_counts()
print("Counts:", counts)
# Decision: if result all zeros -> constant, else balanced
if list(counts.keys())[0] == '0'*n and counts[list(counts.keys())[0]] / sum(counts.values()) > 0.99:
    print("→ Classified CONSTANT")
else:
    print("→ Classified BALANCED")


     ┌───┐┌───┐     ┌─┐                
q_0: ┤ H ├┤ H ├─────┤M├────────────────
     ├───┤├───┤     └╥┘┌─┐             
q_1: ┤ H ├┤ H ├──────╫─┤M├─────────────
     ├───┤├───┤      ║ └╥┘┌───┐┌───┐┌─┐
q_2: ┤ H ├┤ X ├──■───╫──╫─┤ X ├┤ H ├┤M├
     ├───┤├───┤┌─┴─┐ ║  ║ └───┘└───┘└╥┘
q_3: ┤ X ├┤ H ├┤ X ├─╫──╫────────────╫─
     └───┘└───┘└───┘ ║  ║            ║ 
c: 3/════════════════╩══╩════════════╩═
                     0  1            2 
Counts: {'100': 1024}
→ Classified BALANCED


In [4]:
# Task 2: Run Deutsch-Jozsa for different n and inspect circuit depth & counts
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, transpile
from qiskit_aer import AerSimulator

def oracle_balanced_parity(qc, inputs, ancilla):
    # balanced parity oracle: ancilla flipped when parity(input)==1
    for q in inputs:
        qc.cx(q, ancilla)

def deutsch_jozsa_circuit_generic(n, oracle_func):
    qreg = QuantumRegister(n+1, "q")
    creg = ClassicalRegister(n, "c")
    qc = QuantumCircuit(qreg, creg)
    inputs = list(range(n))
    ancilla = n
    qc.x(ancilla)
    qc.h(qreg)
    oracle_func(qc, inputs, ancilla)
    qc.h(inputs)
    qc.measure(inputs, creg)
    return qc

sim = AerSimulator()
for n in [2, 4, 5]:
    qc = deutsch_jozsa_circuit_generic(n, oracle_balanced_parity)
    t_qc = transpile(qc, sim)
    print(f"\n--- n = {n} ---")
    print("Gate depth:", t_qc.depth())
    # Optional: print textual circuit (avoid mpl drawer issues)
    print(qc.draw(fold=-1))
    counts = sim.run(t_qc).result().get_counts()
    print("Counts:", counts)
    # Interpretation
    if list(counts.keys())[0] == '0'*n and counts[list(counts.keys())[0]] / sum(counts.values()) > 0.99:
        print("→ Classified CONSTANT")
    else:
        print("→ Classified BALANCED")



--- n = 2 ---
Gate depth: 5
     ┌───┐          ┌───┐     ┌─┐   
q_0: ┤ H ├───────■──┤ H ├─────┤M├───
     ├───┤       │  └───┘┌───┐└╥┘┌─┐
q_1: ┤ H ├───────┼────■──┤ H ├─╫─┤M├
     ├───┤┌───┐┌─┴─┐┌─┴─┐└───┘ ║ └╥┘
q_2: ┤ X ├┤ H ├┤ X ├┤ X ├──────╫──╫─
     └───┘└───┘└───┘└───┘      ║  ║ 
c: 2/══════════════════════════╩══╩═
                               0  1 
Counts: {'11': 1024}
→ Classified BALANCED

--- n = 4 ---
Gate depth: 7
     ┌───┐          ┌───┐     ┌─┐                   
q_0: ┤ H ├───────■──┤ H ├─────┤M├───────────────────
     ├───┤       │  └───┘┌───┐└╥┘     ┌─┐           
q_1: ┤ H ├───────┼────■──┤ H ├─╫──────┤M├───────────
     ├───┤       │    │  └───┘ ║ ┌───┐└╥┘     ┌─┐   
q_2: ┤ H ├───────┼────┼────■───╫─┤ H ├─╫──────┤M├───
     ├───┤       │    │    │   ║ └───┘ ║ ┌───┐└╥┘┌─┐
q_3: ┤ H ├───────┼────┼────┼───╫───■───╫─┤ H ├─╫─┤M├
     ├───┤┌───┐┌─┴─┐┌─┴─┐┌─┴─┐ ║ ┌─┴─┐ ║ └───┘ ║ └╥┘
q_4: ┤ X ├┤ H ├┤ X ├┤ X ├┤ X ├─╫─┤ X ├─╫───────╫──╫─
     └───┘└───┘└───┘└───┘└───┘ ║ └──

In [5]:
# Task 3: Noise simulation for Deutsch-Jozsa
from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator, noise
from qiskit.visualization import plot_histogram
import matplotlib.pyplot as plt

# Construct example DJ circuit (n=3, parity balanced)
def build_dj_parity(n):
    from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit
    qreg = QuantumRegister(n+1, "q")
    creg = ClassicalRegister(n, "c")
    qc = QuantumCircuit(qreg, creg)
    inputs = list(range(n)); ancilla = n
    qc.x(ancilla); qc.h(qreg)
    for q in inputs: qc.cx(q, ancilla)  # parity oracle
    qc.h(inputs); qc.measure(inputs, creg)
    return qc

n = 3
qc = build_dj_parity(n)

# Ideal run
sim = AerSimulator()
compiled = transpile(qc, sim)
ideal_counts = sim.run(compiled).result().get_counts()

# Build noise model: 1-qubit and 2-qubit depolarizing errors
noise_model = noise.NoiseModel()
error_1q = noise.errors.depolarizing_error(0.02, 1)
error_2q = noise.errors.depolarizing_error(0.06, 2)
noise_model.add_all_qubit_quantum_error(error_1q, ['x', 'h'])
noise_model.add_all_qubit_quantum_error(error_2q, ['cx'])

# Noisy run
noisy_sim = AerSimulator(noise_model=noise_model)
noisy_counts = noisy_sim.run(compiled).result().get_counts()

print("Ideal counts:", ideal_counts)
print("Noisy counts:", noisy_counts)

# Plot histogram (safe)
try:
    plot_histogram([ideal_counts, noisy_counts], legend=['Ideal','Noisy'])
    plt.show()
except Exception:
    print("Plot unavailable — printing counts instead.")
    print("Ideal:", ideal_counts)
    print("Noisy:", noisy_counts)


Ideal counts: {'111': 1024}
Noisy counts: {'001': 30, '000': 16, '010': 2, '110': 37, '101': 30, '011': 63, '111': 846}


In [12]:
# Task 4 — Run Deutsch–Jozsa Algorithm on IBM Quantum or Simulator (Safe Version)
# Works even if no IBM Quantum account is configured

from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, transpile
from qiskit_aer import AerSimulator
from qiskit_ibm_runtime import QiskitRuntimeService, Sampler, Options
from qiskit_ibm_runtime.exceptions import IBMNotAuthorizedError
from qiskit.visualization import plot_histogram
import matplotlib.pyplot as plt

# --- Oracle (balanced parity) ---
def oracle_balanced_parity(qc, inputs, ancilla):
    for q in inputs:
        qc.cx(q, ancilla)

# --- Deutsch–Jozsa circuit builder ---
def deutsch_jozsa_circuit(n, oracle_func):
    qreg = QuantumRegister(n + 1, "q")
    creg = ClassicalRegister(n, "c")
    qc = QuantumCircuit(qreg, creg)
    ancilla = n

    qc.x(ancilla)
    qc.h(range(n + 1))
    oracle_func(qc, list(range(n)), ancilla)
    qc.h(range(n))
    qc.measure(range(n), creg)
    return qc


# --- Main Execution ---
n = 2  # change this to try more input qubits
qc = deutsch_jozsa_circuit(n, oracle_balanced_parity)

print("Deutsch–Jozsa Circuit:")
print(qc.draw(fold=-1))

# --- Try to connect to IBM Quantum ---
try:
    service = QiskitRuntimeService(channel="ibm_quantum")  # requires saved account
    backend = service.least_busy(simulator=False, operational=True)
    print(f"\n✅ Running on real IBM backend: {backend.name}")

    qc_t = transpile(qc, backend)
    options = Options(resilience_level=0, optimization_level=1)
    sampler = Sampler(backend=backend, options=options)
    job = sampler.run(qc_t)
    result = job.result()
    counts = result.quasi_dists[0]

except Exception as e:
    # fallback to local simulator
    print("\n⚠️ IBM account not found or backend unavailable — running on local AerSimulator.")
    simulator = AerSimulator()
    qc_t = transpile(qc, simulator)
    job = simulator.run(qc_t, shots=1024)
    result = job.result()
    counts = result.get_counts()

# --- Display Results ---
print("\nMeasurement Counts:", counts)
plot_histogram(counts)
plt.show()


Deutsch–Jozsa Circuit:
     ┌───┐          ┌───┐     ┌─┐   
q_0: ┤ H ├───────■──┤ H ├─────┤M├───
     ├───┤       │  └───┘┌───┐└╥┘┌─┐
q_1: ┤ H ├───────┼────■──┤ H ├─╫─┤M├
     ├───┤┌───┐┌─┴─┐┌─┴─┐└───┘ ║ └╥┘
q_2: ┤ X ├┤ H ├┤ X ├┤ X ├──────╫──╫─
     └───┘└───┘└───┘└───┘      ║  ║ 
c: 2/══════════════════════════╩══╩═
                               0  1 

⚠️ IBM account not found or backend unavailable — running on local AerSimulator.

Measurement Counts: {'11': 1024}


In [9]:
# Task 5: Print oracle circuit definition and (optionally) unitary (for small n)
from qiskit import QuantumCircuit
from qiskit.quantum_info import Operator

# Example: oracle that flips ancilla when MSB==0 (from Task 1)
def oracle_flip_if_msb_zero(qc, inputs, ancilla):
    msb = inputs[-1]
    qc.x(msb)
    qc.cx(msb, ancilla)
    qc.x(msb)

# Build oracle as separate QuantumCircuit (n=2 example)
n = 2
qc_oracle = QuantumCircuit(n+1)
oracle_flip_if_msb_zero(qc_oracle, list(range(n)), n)

# Print the gate/circuit definition (text)
print("Oracle circuit (text):")
print(qc_oracle.draw(fold=-1))

# Convert to a gate and print its definition (subcircuit)
gate = qc_oracle.to_gate(label="oracle_msb0")
print("\nOracle gate definition:")
print(gate.definition.draw(fold=-1))

# Optional: print full unitary matrix for small n
try:
    op = Operator(qc_oracle)
    print("\nOracle unitary matrix (shape {}):\n".format(op.data.shape))
    print(op.data)
except Exception as e:
    print("Cannot compute unitary (matrix too large?), error:", e)

# Explanation (example):
# - The oracle acts as |x>|y> -> |x>|y ⊕ f(x)>. For this oracle f(x)=1 when MSB==0.
# - The circuit uses X on MSB to transform condition MSB==0 to MSB==1, then a CNOT
#   controlled by MSB onto ancilla flips ancilla when the (original) MSB was 0.


Oracle circuit (text):
                    
q_0: ───────────────
     ┌───┐     ┌───┐
q_1: ┤ X ├──■──┤ X ├
     └───┘┌─┴─┐└───┘
q_2: ─────┤ X ├─────
          └───┘     

Oracle gate definition:
                    
q_0: ───────────────
     ┌───┐     ┌───┐
q_1: ┤ X ├──■──┤ X ├
     └───┘┌─┴─┐└───┘
q_2: ─────┤ X ├─────
          └───┘     

Oracle unitary matrix (shape (8, 8)):

[[0.+0.j 0.+0.j 0.+0.j 0.+0.j 1.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 1.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 1.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 1.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [1.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 1.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 1.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 1.+0.j]]
